Installing libs 

In [ ]:
# Install the required libraries
from IPython.display import clear_output
try:
  !pip install pystan
  !pip install prophet
  !pip install hvplot
  !pip install holoviews
except:
  print('Error installing required dependencies')
finally:
  clear_output()
  print('Dependencies installed successfully')

In [ ]:
# Import the required libraries and dependencies
import pandas as pd
import holoviews as hv
import hvplot.pandas
import datetime as dt
import numpy as np

from prophet import Prophet

%matplotlib inline

print('Required libraries and dependencies imported')

 Step 1: Find Unusual Patterns in Hourly Google Search Traffic

#### Step 1: Read the search data into a DataFrame, and then slice the data to just the month of May 2020. (During this month, MercadoLibre released its quarterly financial results.) Use hvPlot to visualize the results. Do any unusual patterns exist?

In [ ]:
# Upload the "google_hourly_search_trends.csv" file into Colab, then store in a Pandas DataFrame
# Set the "Date" column as the Datetime Index.

from google.colab import files
uploaded = files.upload()

df_mercado_trends = pd.read_csv('google_hourly_search_trends.csv', 
    infer_datetime_format=True, 
    parse_dates=True, 
    index_col='Date'   
    )

# Review the first and last five rows of the DataFrame
print('Review the first five rows of the DataFrame')
display(df_mercado_trends.head())

print('Review the last five rows of the DataFrame')
display(df_mercado_trends.tail())

In [ ]:
# Review the data types of the DataFrame using the info function
print('Data Types of the columns in the DataFrame')
df_mercado_trends.info()

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Slice the DataFrame to just the month of May 2020
df_may_2020 = df_mercado_trends.loc['2020-05']

# Use hvPlot to visualize the data for May 2020
df_may_2020.hvplot(
    title='HvPlot - visualizing the Google hourly search trend data for May 2020',
    hover_color = 'magenta'
    )

#### Step 2: Calculate the total search traffic for the month, and then compare the value to the monthly median across all months. Did the Google search traffic increase during the month that MercadoLibre released its financial results?

In [ ]:
# Calculate the sum of the total search traffic for May 2020
traffic_may_2020 = df_may_2020.sum()

# View the traffic_may_2020 value
traffic_may_2020

In [ ]:
# Calculate the monthly median search traffic across all months 
# Group the DataFrame by index year and then index month, chain the sum and then the median functions
groupby_levels = [df_mercado_trends.index.year, df_mercado_trends.index.month]
median_monthly_traffic = df_mercado_trends.groupby(by = groupby_levels).sum().median()

# View the median_monthly_traffic value
print('View the monthly median search traffic across all months')
median_monthly_traffic

In [ ]:
# Compare the seach traffic for the month of May 2020 to the overall monthly median value
# Compare the seach traffic for the month of May 2020 to the overall monthly median value
print(f'\n Total search traffic for May 2020 is: {float(traffic_may_2020)}'
      f'\n\n Median monthly search traffic across all months is: {float(median_monthly_traffic)}'
      f'\n\n Percentage search traffic in May 2020 as compared all other months is: {round(float((traffic_may_2020 / median_monthly_traffic)-1)*100, 2)}%')


In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Group the hourly search data to plot (use hvPlot) the average traffic by the day of week 
groupby_weekdays = df_mercado_trends.index.weekday
grouped_daily_mercado_trends = df_mercado_trends.groupby(groupby_weekdays).mean()
grouped_daily_mercado_trends.hvplot(
    title = 'Average traffic by the day of the week, grouped hourly',
    color = 'green',
    hover_color = 'magenta'
)

#### Step 2: Using hvPlot, visualize this traffic as a heatmap, referencing the `index.hour` as the x-axis and the `index.dayofweek` as the y-axis. Does any day-of-week effect that you observe concentrate in just a few hours of that day?

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the hour of the day and day of week search traffic as a heatmap.
df_mercado_trends.hvplot.heatmap(title = 'Heatmap showing visualize the hour of the day and day of week search traffic', x= "index.hour", y="index.dayofweek", C="Search Trends", cmap= 'spectral').aggregate(function = np.sum)

##### Answer the following question:

**Question:** Does any day-of-week effect that you observe concentrate in just a few hours of that day?

**Answer:**
* Hour wise, we see higher concentrations especially during 12 - 01 AM and 10 - 11 PM. Means, search quite active for 4 hours. And this higher concentration of searches are pretty uniform from days 0 till day 5 of the week.
* The above search pattern of hours/days shows that the MercadoLibre ecommerce website has online users mostly from non-Latin America region, possibly Europe or Mid-East Asia.



#### Step 3: Group the search data by the week of the year. Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Group the hourly search data to plot (use hvPlot) the average traffic by the week of the year
# The hourly search data is contained the DataFrame: df_mercado_trends
groupby_weeks = df_mercado_trends.index.weekofyear
grouped_weekly_mercado_trends = df_mercado_trends.groupby(groupby_weeks).mean()
grouped_weekly_mercado_trends.hvplot(
    hover_color = 'magenta',
    color = 'blue',
    title = 'Average search traffic by the week of the year'   
)

#### Step 1: Read in and plot the stock price data. Concatenate the stock price data to the search data in a single DataFrame.

In [ ]:
# Upload the "mercado_stock_price.csv" file into Colab, then store in a Pandas DataFrame
# Set the "date" column as the Datetime Index.
from google.colab import files
uploaded = files.upload()

df_mercado_stock = pd.read_csv(
    'mercado_stock_price.csv', 
    infer_datetime_format = True, 
    parse_dates = True, 
    index_col ='date'   
)

# View the first and last five rows of the DataFrame
print(f'View the first 5 rows of the DataFrame created from mercado_stock_price.csv')
display(df_mercado_stock.head())

print(f'View the last 5 rows of the DataFrame created from mercado_stock_price.csv')
display(df_mercado_stock.tail())

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the closing price of the df_mercado_stock DataFrame
df_mercado_stock.hvplot(
    color = 'green',
    hover_color = 'magenta',
    rot = 90,
    title = 'Mercado stock closing price' 
)

In [ ]:
# Concatenate the df_mercado_stock DataFrame with the df_mercado_trends DataFrame
# Concatenate the DataFrame by columns (axis=1), and drop and rows with only one column of data
concat_df = pd.concat([df_mercado_stock, df_mercado_trends], axis=1)
print('See what is the NaN column like')
display(concat_df.head(10))

mercado_stock_trends_df = concat_df.dropna()

# View the first and last five rows of the DataFrame
print('\n View the first 5 rows of the concatenated mercado DataFrame')
display(mercado_stock_trends_df.head(5))

print('\n View the last 5 rows of the concatenated mercado DataFrame')
mercado_stock_trends_df.tail(5)

#### Step 2: Market events emerged during the year of 2020 that many companies found difficult. But, after the initial shock to global financial markets, new customers and revenue increased for e-commerce platforms. Slice the data to just the first half of 2020 (`2020-01` to `2020-06` in the DataFrame), and then use hvPlot to plot the data. Do both time series indicate a common trend that’s consistent with this narrative?

In [ ]:
# For the combined dataframe, slice to just the first half of 2020 (2020-01 through 2020-06) 
first_half_2020 = mercado_stock_trends_df.loc['2020-01':'2020-06']

# View the first and last five rows of first_half_2020 DataFrame
print('Sliced view of the first 5 rows of first_half_2020 DataFrame')
display(first_half_2020.head(5))

print('\n Sliced view of the first 5 rows of first_half_2020 DataFrame')
display(first_half_2020.tail(5))

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the close and Search Trends data
# Plot each column on a separate axes using the following syntax
# `hvplot(shared_axes=False, subplots=True).cols(1)`
print('Plot each column (close and search trends) on a separate axes')
first_half_2020.hvplot(color = 'blue', shared_axes = False, subplots = True, rot = 45).cols(1)

##### Answer the following question:

**Question:** Do both time series indicate a common trend that’s consistent with this narrative?

**Answer:**
* Yes, there is a common trend between the close price and volume of search trends during the first half of the year. The drop in close price is also visible between the period of 2020-02-25 and 2020-03-12 across both the plots of close vs. search trends.

#### Step 3: Create a new column in the DataFrame named “Lagged Search Trends” that offsets, or shifts, the search traffic by one hour. Create two additional columns:

* “Stock Volatility”, which holds an exponentially weighted four-hour rolling average of the company’s stock volatility

* “Hourly Stock Return”, which holds the percent change of the company's stock price on an hourly basis


In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Lagged Search Trends
# This column should shift the Search Trends information by one hour
mercado_stock_trends_df['Lagged Search Trends'] = mercado_stock_trends_df['Search Trends'].shift(periods = 1)

print('View the newly added column Lagged Search Trends after a period shift of 1 hour')
display(mercado_stock_trends_df.head(10))

In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Stock Volatility
# This column should calculate the standard deviation of the closing stock price return data over a 4 period rolling window

mercado_stock_trends_df['Stock Volatility'] = mercado_stock_trends_df['close'].pct_change().rolling(window = 4).std()

print('View the calculated standard deviation of the closing stock price return data over a 4 period rolling window in a new column Stock Volatility')
display(mercado_stock_trends_df.head(10))

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the stock volatility
mercado_stock_trends_df['Stock Volatility'].hvplot(
    color = 'green',
    hover_color = 'blue',
    rot = 45
)

**Solution Note:** Note how volatility spiked, and tended to stay high, during the first half of 2020. This is a common characteristic of volatility in stock returns worldwide: high volatility days tend to be followed by yet more high volatility days. When it rains, it pours.

In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Hourly Stock Return
# This column should calculate hourly return percentage of the closing price
mercado_stock_trends_df['Hourly Stock Return'] = mercado_stock_trends_df['close'].pct_change()

In [ ]:
# View the first and last five rows of the mercado_stock_trends_df DataFrame
print('Hourly pct_change of the closing price in a new column Hourly Stock Return')
display(mercado_stock_trends_df.head())

display(mercado_stock_trends_df.head())

#### Step 4: Review the time series correlation, and then answer the following question: Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

In [ ]:
# Construct correlation table of Stock Volatility, Lagged Search Trends, and Hourly Stock Return
print('Correlation table of Stock Volatility, Lagged Search Trends, and Hourly Stock Return values')
mercado_stock_trends_df[['Stock Volatility', 'Lagged Search Trends', 'Hourly Stock Return']].corr()

#### Step 1: Set up the Google search data for a Prophet forecasting model.

In [ ]:
# Using the df_mercado_trends DataFrame, reset the index so the date information is no longer the index
mercado_prophet_df = df_mercado_trends.reset_index()

# Label the columns ds and y so that the syntax is recognized by Prophet
mercado_prophet_df.columns = ['ds', 'y']

# Drop an NaN values from the prophet_df DataFrame
mercado_prophet_df = mercado_prophet_df.dropna()

# View the first and last five rows of the mercado_prophet_df DataFrame
print('View the first and last 5 rows of the mercado_prophet_df DataFrame after resetting the date index')
display(mercado_prophet_df.head())

display(mercado_prophet_df.tail())

In [ ]:
# Call the Prophet function, store as an object
model_mercado_trends = Prophet()

In [ ]:
# Fit the time-series model.
model_mercado_trends.fit(mercado_prophet_df)

In [ ]:
# Create a future dataframe to hold predictions
# Make the prediction go out as far as 2000 hours (approx 80 days)
future_mercado_trends = model_mercado_trends.make_future_dataframe(periods = 2000, freq = 'H')

# View the last five rows of the future_mercado_trends DataFrame
print('View the last 5 rows of the future_mercado_trends DataFrame')
future_mercado_trends.tail()


In [ ]:
# Make the predictions for the trend data using the future_mercado_trends DataFrame
forecast_mercado_trends = model_mercado_trends.predict(future_mercado_trends)

# Display the first five rows of the forecast_mercado_trends DataFrame
print('Display the first five rows of the forecast_mercado_trends DataFrame')
display(forecast_mercado_trends.head())

display(forecast_mercado_trends.tail())

#### Step 2: After estimating the model, plot the forecast. How's the near-term forecast for the popularity of MercadoLibre?

In [ ]:
# Plot the Prophet predictions for the Mercado trends data
model_mercado_trends.plot(forecast_mercado_trends)

##### Answer the following question:

In [ ]:
# Set the index in the forecast_mercado_trends DataFrame to the ds datetime column
forecast_mercado_trends = forecast_mercado_trends.set_index('ds')

# View the only the yhat,yhat_lower and yhat_upper columns from the DataFrame
forecast_mercado_trends[['yhat', 'yhat_lower', 'yhat_upper']]

Solutions Note: `yhat` represents the most likely (average) forecast, whereas `yhat_lower` and `yhat_upper` represents the worst and best case prediction (based on what are known as 95% confidence intervals).

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# From the forecast_mercado_trends DataFrame, use hvPlot to visualize
#  the yhat, yhat_lower, and yhat_upper columns over the last 2000 hours
print('Visualize the Mercado trends from the prediction dataset \n')
forecast_mercado_trends[['yhat', 'yhat_lower', 'yhat_upper']].iloc[-200:,:].hvplot(
    title = 'Forecast - Mercado Prediction Trends of yhat, yhat lower, and yhat upper',
    hover_color = 'magenta',
    rot = 45
    )

In [ ]:
# Reset the index in the forecast_mercado_trends DataFrame
forecast_mercado_trends = forecast_mercado_trends.reset_index()

# Use the plot_components function to visualize the forecast results 
# for the forecast_canada DataFrame
print('Plot the individual time series components of the model \n')
figures_mercado_trends = model_mercado_trends.plot_components(forecast_mercado_trends)

##### Answer the following questions:

**Question:** What time of day exhibits the greatest popularity?

**Answer:**
* The late evening hours around 11 PM and 12 AM exhibits the greatest popularity
* The increase in popularity trend begins after 9 AM.

**Question:** Which day of week gets the most search traffic? 
   
**Answer:**
* It's Tuesday

**Question:** What's the lowest point for search traffic in the calendar year?

**Answer:**
* It's towards September last week and October month


#### Step 1: Read in the daily historical sales (that is, revenue) figures, and then apply a Prophet model to the data.

In [ ]:
# Upload the "mercado_daily_revenue.csv" file into Colab, then store in a Pandas DataFrame
# Set the "date" column as the DatetimeIndex
# Sales are quoted in millions of US dollars
from google.colab import files
uploaded = files.upload()

df_mercado_sales = pd.read_csv(
    'mercado_daily_revenue.csv', 
    infer_datetime_format=True, 
    parse_dates=True, 
    index_col='date'   
    )

# Review the DataFrame
df_mercado_sales

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the daily sales figures 
df_mercado_sales.hvplot(
    title = 'Mercado daily sales revenue',
    color = 'gray',
    hover_color = 'green'
)

In [ ]:
# Apply a Facebook Prophet model to the data.

# Set up the dataframe in the neccessary format:
# Reset the index so that date becomes a column in the DataFrame
mercado_sales_prophet_df = df_mercado_sales.reset_index()

# Adjust the columns names to the Prophet syntax
mercado_sales_prophet_df.columns = ['ds', 'y']

# Visualize the DataFrame
print('Date ds column index reset')
mercado_sales_prophet_df

In [ ]:
# Create the model
mercado_sales_prophet_model = Prophet()

# Fit the model
mercado_sales_prophet_model.fit(mercado_sales_prophet_df)

In [ ]:
# Predict sales for 90 days (1 quarter) out into the future.

# Start by making a future dataframe
mercado_sales_prophet_future = mercado_sales_prophet_model.make_future_dataframe(periods = 90, freq = 'D')

# Display the last five rows of the future DataFrame
display(mercado_sales_prophet_future.tail())


In [ ]:
# Make predictions for the sales each day over the next quarter
mercado_sales_prophet_forecast = mercado_sales_prophet_model.predict(mercado_sales_prophet_future)

# Display the first 5 rows of the resulting DataFrame
mercado_sales_prophet_forecast.head()


In [ ]:
# Use the plot_components function to analyze seasonal patterns in the company's revenue
mercado_sales_prophet_model.plot_components(mercado_sales_prophet_forecast)

##### Answer the following question:

**Question:** For example, what are the peak revenue days? (Mondays? Fridays? Something else?)

**Answer:**
* Wednesday shows the highest peak in the revenue. Saturday shows the bottom point in the revenue.

#### Step 3: Produce a sales forecast for the finance group. Give them a number for the expected total sales in the next quarter. Include the best- and worst-case scenarios to help them make better plans.

In [ ]:
# Plot the predictions for the Mercado sales
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

forecast_mercado_sales= mercado_sales_prophet_forecast.set_index('ds')
forecast_mercado_sales[['yhat', 'yhat_lower', 'yhat_upper']].iloc[-90:,:].hvplot(
    hover_color = 'magenta',
    title = 'Predictions for Mercado Sales'
)